# 1st Part 

Import all the required Libraries

In [0]:
import requests
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from bs4 import BeautifulSoup
import numpy as np
from geopy.geocoders import Nominatim
import folium
import json
from sklearn.cluster import KMeans

Extract table from the given url and transform it to Data Frame

In [0]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
res = requests.get(url)
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
df[0].to_json(orient='records')
 

'[{"Postal code":"M1A","Borough":"Not assigned","Neighborhood":null},{"Postal code":"M2A","Borough":"Not assigned","Neighborhood":null},{"Postal code":"M3A","Borough":"North York","Neighborhood":"Parkwoods"},{"Postal code":"M4A","Borough":"North York","Neighborhood":"Victoria Village"},{"Postal code":"M5A","Borough":"Downtown Toronto","Neighborhood":"Regent Park \\/ Harbourfront"},{"Postal code":"M6A","Borough":"North York","Neighborhood":"Lawrence Manor \\/ Lawrence Heights"},{"Postal code":"M7A","Borough":"Downtown Toronto","Neighborhood":"Queen\'s Park \\/ Ontario Provincial Government"},{"Postal code":"M8A","Borough":"Not assigned","Neighborhood":null},{"Postal code":"M9A","Borough":"Etobicoke","Neighborhood":"Islington Avenue"},{"Postal code":"M1B","Borough":"Scarborough","Neighborhood":"Malvern \\/ Rouge"},{"Postal code":"M2B","Borough":"Not assigned","Neighborhood":null},{"Postal code":"M3B","Borough":"North York","Neighborhood":"Don Mills"},{"Postal code":"M4B","Borough":"East 

In [0]:
type(df[0])

pandas.core.frame.DataFrame

In [0]:
df = df[0]

In [0]:
df.columns

Index(['Postal code', 'Borough', 'Neighborhood'], dtype='object')

Cleaning the data (dropping the not assigned Borough entries and merging neighborhoods with same Postalcode ) 

In [0]:
indexNames = df[ df['Borough'] =='Not assigned'].index
df.drop(indexNames , inplace=True)
df.loc[df['Neighborhood'] =='Not assigned' , 'Neighborhood'] = df['Borough']
result = df.groupby(['Postal code','Borough'], sort=False).agg( ', '.join)
df_new=result.reset_index()

In [47]:
df_new.head(10)

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,Malvern / Rouge
7,M3B,North York,Don Mills
8,M4B,East York,Parkview Hill / Woodbine Gardens
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [0]:
df_new.shape

(103, 3)

# 2nd Part


Using the geographical location data from the given url merge the contents based on Postal Code 

In [0]:
url="http://cocl.us/Geospatial_data"
import io
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))
c.columns = ['Postal code', 'Latitude', 'Longitude']
df = pd.merge(c, df_new, on='Postal code')

# reorder column names and show the dataframe
df = df[['Postal code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]

In [59]:
df.head(5)

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
